In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

import time
import random
import re
from itertools import product

import pandas as pd

In [2]:
def search_matrix(rooms=None, floor=None, is_apartment=None, not_last_floor=None):
    '''
    функция служит для создания матрицы поиска, которая будет использоваться 
    парсером. Таким образом можно существенно увеличить количество объявлений.
    
    Принимает на вход:
    rooms - целое, список - количество комнат
    floor - целое, список - этаж
    is_apartment - boolean - является ли объект апартементами
    last_floor - boolean - является ли этаж последним
    
    Возвращает:
    список кортежей с заданными параметрами. 
    Индексация кортежей 
    [0] - количество комнат
    [1] - этаж
    [2] - апартаменты True / False
    [3] - последний этаж True / False
    '''
    if not rooms:
        rooms = [i for i in range(1,10)]
    elif type(rooms) == list:
        pass
    else:
        rooms = [rooms]
    
    if not floor:
        floor = [i for i in range(1,50)]
    elif type(floor) == list:
        pass
    else:
        floor = [floor]
    
    if not is_apartment:
        if is_apartment == False:
            is_apartment = [is_apartment]
        else:
            is_apartment = [True, False]
    else:
        is_apartment = [is_apartment]
        
    if not not_last_floor:
        if not_last_floor == False:
            not_last_floor = [not_last_floor]
        else:
            not_last_floor = [True, False]
    else:
        not_last_floor = [not_last_floor]
        
    return list(product(rooms, floor, is_apartment, not_last_floor))

In [3]:
def search_url(params):
    base_link = f'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2'
    
    rooms = params[0]
    floor = params[1]
    is_apartment = params[2]
    not_last_floor = params[3]
    search_url = base_link + \
    f'&floornl={not_last_floor*1}' + \
    f'&maxfloor={floor}&minfloor={floor}' + \
    '&offer_type=flat'
    
    if is_apartment:
        search_url = search_url[:28] + 'apartment=1&' + search_url[28:]
    else:
        search_url = search_url + '&only_flat=1'
    
    search_url += f'&p=1&region=1&room{rooms}=1'
    return search_url

In [4]:
def parse_cian_with_defined_params(rooms=1, floor=1, is_apartment=None, not_last_floor=None):
    '''
    Ходит по страницам циана и собирает нужную информацию
    
    Если не задавать параметры, то разделяет самостоятельно объявления по параметрам комнатности и этажности
    При этом для каждой ячейки этой матрицы собирает максимум 1512 объявлений (ограничение циана)
    
    Возвращает - словарь с объявлениями. Ключ - id объявления.
    '''
    
    driver = webdriver.Chrome()
    #задержки можно убрать на свой страх и риск
    time.sleep(2)
    result = {}
    
    grid = search_matrix(rooms, floor, is_apartment, not_last_floor)
    
    print(f'Всего запросов: {len(grid)}')
    for count, params in enumerate(grid):
        page = 1
        url = search_url(params)
        driver.get(url)

        try:
            while True:
                #задержки можно убрать на свой страх и риск
                time.sleep(1)
                soup = BeautifulSoup(driver.page_source,'lxml')
                print(f'\rОбрабатывается запрос: {count+1}, Уникальных записей в словаре: {len(result)}', end='')
                #тут карточки всех объектов недвиги
                for i in soup.find_all('div','_93444fe79c--card--ibP42'):
                    # i - карточка отдельного объекта
        
                    link = i.find('a', '_93444fe79c--link--eoxce')['href']
                    ad_id = link.split('/')[-2]
                    result[ad_id] = {}
                    result[ad_id]['link'] = link
                    
                    result[ad_id]['rooms'] = params[0]
                    result[ad_id]['floor'] = params[1]
                    result[ad_id]['is_apartment'] = params[2]
                    result[ad_id]['not_last_floor'] = params[3]
                
                    title = i.find('span',{'data-mark':'OfferTitle'})
                    result[ad_id]['title'] = title.text
        
                    try:
                        subtitle = i.find('span',{'data-mark':'OfferSubtitle'})
                        result[ad_id]['subtitle'] = subtitle.text
                    except:
                        pass
        
                    try:
                        deadline = i.find('span', {'data-mark':'Deadline'})
                        result[ad_id]['deadline'] = deadline.text
                    except:
                        pass
        
                    try:
                        metro = i.find('a','_93444fe79c--link--BwwJO')
                        result[ad_id]['metro'] = metro.text
                        metro_remote = i.find('div','_93444fe79c--remoteness--q8IXp')
                        result[ad_id]['metro_remote'] = metro_remote.text
        
                    except:
                        pass
        
                    geo = i.find('div','_93444fe79c--labels--L8WyJ')
                    result[ad_id]['geo'] = geo.text
        
                    main_price = i.find('span',{'data-mark':'MainPrice'})
                    result[ad_id]['main_price'] = int(re.sub(' ', '',main_price.text[:-2]))
                    if main_price.text[-1] == '₽':
                        currency = 'rouble'
                    else:
                        currency = 'other'
                    result[ad_id]['currency'] = currency
        
                    price_for_sq_m = i.find('p',{'data-mark':'PriceInfo'})
                    result[ad_id]['price_for_sq_m'] = int(re.sub(' ', '',price_for_sq_m.text[:-5]))
        
                
                #проверяем наличие заблокированных кнопок
                if soup.find_all('button',{'data-name':'PaginationButton'}):
                    #заблокированные кнопки есть, если последняя это "Дальше", то начинаем поиск по новым параметрам
                    if soup.find_all('button',{'data-name':'PaginationButton'})[-1].text == 'Дальше':
                        break
                    #в ином случае заблоченные кнопки есть, но "Дальше" активна, переворачиваем страницу
                    else:
                        page += 1
                        url = re.sub(f'p={page-1}', f'p={page}', url)
                        driver.get(url)
                        #задержки можно убрать на свой страх и риск
                        time.sleep(1)
                        
                #если кнопок нет, то начинаем поиск по новым параметрам
                else:
                    break
        #ошибка или ручное прерывание            
        except:
            print('\nПоиск завершился с ошибкой')
            return result
    #основной цикл закончен, выводим результат
    print('\nПоиск по сетке завершен')
    driver.quit()
    return result    
            

In [5]:
parsed_data = parse_cian_with_defined_params()

Всего запросов: 4
Обрабатывается запрос: 4, Уникальных записей в словаре: 1178
Поиск по сетке завершен


In [6]:
df = pd.DataFrame(parsed_data).T
df

,link,rooms,floor,is_apartment,not_last_floor,title,subtitle,deadline,metro,metro_remote,geo,main_price,currency,price_for_sq_m
289464684,https://www.cian.ru/sale/flat/289464684/,1,1,True,True,"1-комн. апарт., 39,39 м², 1/5 этаж",Сдача корпуса 4 кв. 2024,сдача ГК: 3 кв. 2024 года,Площадь Революции,2 минуты пешком,"Москва, ЦАО, р-н Тверской, м. Площадь Революци...",80870000,rouble,2053059
283849683,https://www.cian.ru/sale/flat/283849683/,1,1,True,True,"Новая студия 12,5 кв.м","1-комн. апарт., 12,5 м², 1/5 этаж",NaN,Волжская,5 минут пешком,"Москва, ЮВАО, р-н Текстильщики, м. Волжская, у...",4000000,rouble,320000
288775559,https://www.cian.ru/sale/flat/288775559/,1,1,True,True,"1-к. пом., 29,66 м², 1/3 этаж",Сдача корпуса 2 кв. 2025,сдача ГК: 3 кв. 2026 года,Новокузнецкая,2 минуты пешком,"Москва, ЦАО, р-н Замоскворечье, м. Новокузнецк...",24867537,rouble,838420
288719837,https://www.cian.ru/sale/flat/288719837/,1,1,True,True,Новая студия на Западе Москвы,"1-комн. апарт., 14,4 м², 1/17 этаж",NaN,Кунцевская,5 минут на транспорте,"Москва, ЗАО, р-н Можайский, м. Кунцевская, Дор...",4100000,rouble,284722
270246372,https://www.cian.ru/sale/flat/270246372/,1,1,True,True,"1-комн. апарт., 37,3 м², 1/8 этаж",Секция 1 • Сдача корпуса 2 кв. 2024,сдача ГК: 2 кв. 2024 года,Петровский Парк,5 минут пешком,"Москва, САО, р-н Савеловский, м. Петровский Па...",14346459,rouble,384624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288575973,https://www.cian.ru/sale/flat/288575973/,1,1,False,False,"1-комн. кв., 33,1 м², 3/9 этаж",NaN,NaN,Бунинская аллея,10 минут на транспорте,"Москва, НАО (Новомосковский), м. Щербинка, Щер...",6500000,rouble,196375
286379364,https://www.cian.ru/sale/flat/286379364/,1,1,False,False,"1-комн. кв., 40,7 м², 11/17 этаж",NaN,NaN,Щербинка,4 минуты на транспорте,"Москва, НАО (Новомосковский), м. Щербинка, Щер...",7000000,rouble,171990
286452179,https://www.cian.ru/sale/flat/286452179/,1,1,False,False,"Студия, 23,24 м², 4/15 этаж",Сдача корпуса 4 кв. 2024,сдача ГК: 4 кв. 2024 года,Силикатная,4 минуты на транспорте,"Москва, НАО (Новомосковский), м. Силикатная, Р...",4929752,rouble,212124
278940666,https://www.cian.ru/sale/flat/278940666/,1,1,False,False,"1-комн. кв., 42 м², 3/17 этаж",NaN,NaN,Щербинка,4 минуты на транспорте,"Москва, ЮЗАО, р-н Южное Бутово, м. Щербинка, у...",7000000,rouble,166667


In [ ]:
parse_all_data = parse_cian_with_defined_params(None, None, None, None)